## Building A Chatbot
We will design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This is a basic chatbot
1. Load the environment variables from the .env file.
2. Store these env variable in a separate variable used further to access the models.
3. Select and load your model. In our case, we are using a Gemma model from chatgroq, langchain.
4. Chatting with chatbot to see if it remembers previous history.

##### In case of multiple users, we need to maintain session and session data for each chat.
5. 

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()   # loading all the environment variable


True

In [4]:
# Groq - AI inference for faster compute and memory
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x114ee8050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x114ee8d70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# Start with basic text
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer")])

# Here I have not passed anything from the SystemMessage. 
# SystemMessage will tell what kind of behaviour the LLM will perform.
# AIMessage : is the response from LLM Model.

AIMessage(content="Hi Divya Gupta, it's nice to meet you!  \n\nBeing a Lead Developer is a great role. What kind of projects do you typically work on?  \n\nI'm happy to chat about anything you'd like, whether it's about your work, technology, or just something else entirely. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 23, 'total_tokens': 92, 'completion_time': 0.125454545, 'prompt_time': 0.002121366, 'queue_time': 0.160019053, 'total_time': 0.127575911}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--da05660c-bfd7-4386-9c05-59d5b27d406b-0', usage_metadata={'input_tokens': 23, 'output_tokens': 69, 'total_tokens': 92})

In [ ]:
# We are giving a list of messages. And giving some previous information.
# The chatbot is remembering the previous information too.

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer"),
        AIMessage(content="Hi Divya Gupta, it's nice to meet you!  \n\nBeing a Lead Developer is a great role. What kind of projects do you typically work on?  \n\nI'm happy to chat about anything you'd like, whether it's about your work, technology, or just something else entirely. 😊\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Divya Gupta, and you are a Lead Developer!  \n\nIs there anything else you'd like to know or discuss? 😊  I'm here to help. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 112, 'total_tokens': 153, 'completion_time': 0.074545455, 'prompt_time': 0.007408926, 'queue_time': 0.161532398, 'total_time': 0.081954381}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a9ee539e-db64-40aa-9f18-a78c3c3df157-0', usage_metadata={'input_tokens': 112, 'output_tokens': 41, 'total_tokens': 153})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory #every next message should be the part of the previous conversation.

store={}

# Distinguish one chat session with the other
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() # whatever chat is happening will go to this session id.
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

# Here : 
# 1. we create a dictonary.
# 2. Then we created a function, which will retrieve, whatever chat history is there for a specific session id.
# 3. Then we are converting/using the RunnableWithMessageHistory, so we can interact with our LM model based on the chat history.


In [11]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response = with_message_history.invoke([HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer")],
    config=config)

In [15]:
response.content

"Hi Divya Gupta!  \n\nIt's great to meet you.  Being a Lead Developer is a challenging and rewarding role. What kind of projects are you currently leading?  \n\nI'm curious to hear about the technologies you're using and the problems you're solving.\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Divya Gupta.  \n\nI remember from our earlier conversation. 😊  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 171, 'total_tokens': 200, 'completion_time': 0.052727273, 'prompt_time': 0.008641406, 'queue_time': 0.16259483400000002, 'total_time': 0.061368679}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b593633-8165-4cfb-8832-ec69b7f64de8-0', usage_metadata={'input_tokens': 171, 'output_tokens': 29, 'total_tokens': 200})

In [17]:
# Till now you are part of seesion id 1. 
# Now lets change the session id and see if your chatbot will be able to remember your conversation

## change the config-->session id
config2={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [18]:
# Feeding information to the session2.
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config2
)
response.content

"Hi John! It's nice to meet you. 😊 \n\nWhat can I do for you today?\n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)
response.content

"Your name is John, that's what you told me! 😊  \n\nIs there anything else I can help you with?\n"